<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [3]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [4]:
raw_data[:10]

'First Citi'

In [5]:
dataset[:10]

['First Citizen:\n',
 'Before we proceed any further, hear me speak.\n',
 '\n',
 'All:\n',
 'Speak, speak.\n',
 '\n',
 'First Citizen:\n',
 'You are all resolved rather to die than to famish?\n',
 '\n',
 'All:\n']

In [6]:
vocab = set(raw_data)

In [7]:
vocab_len = len(vocab)

In [8]:
vocab_len

65

In [9]:
stoi = {char:i for i, char in enumerate(vocab)}

In [10]:
itos = {i:char for i, char in enumerate(vocab)}

In [11]:
import torch

In [12]:
import torch.nn.functional as functional

In [24]:
# make dataset
xs, ys = [], []
for data in dataset:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

2085234

In [14]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([130, 65])

In [32]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([-1.7541e+00,  8.1739e-01, -8.3656e-01, -4.3790e-01,  1.6453e+00,
         4.8561e-01,  1.8303e+00,  1.7494e+00,  3.9286e-01,  1.0130e+00,
        -2.0530e-01,  3.6624e+00, -4.1178e-01,  2.1023e+00,  1.1245e+00,
        -6.0958e-01,  2.4157e+00,  3.2442e-01, -7.2540e-01, -5.1809e-01,
         1.0741e+00, -1.6094e+00, -1.5116e+00,  1.1825e+00, -2.9752e-02,
        -1.5847e+00,  4.5340e-01, -3.1264e-02,  7.9392e-01, -5.1613e-04,
         1.7675e+00, -1.7271e+00,  1.7110e+00, -1.9072e-01, -3.3966e+00,
        -2.1671e+00, -2.0422e-01,  2.1972e-01,  6.5840e-01, -5.4364e-01,
         1.9716e+00, -3.1429e-01, -8.7922e-02, -1.5624e-01,  1.1395e+00,
         1.5801e+00,  1.2501e+00, -9.0533e-01,  2.5734e+00, -8.0956e-02,
        -2.5074e-01,  1.0147e+00,  1.4038e+00, -1.6368e+00,  9.2717e-02,
         2.8855e+00,  1.8129e-01,  1.1702e+00, -8.9394e-01, -1.0003e-01,
        -3.2349e-01, -4.6710e-01,  6.5940e-01, -1.6062e-01,  6.7161e-01],
       grad_fn=<SqueezeBackward4>)

In [26]:
(xenc @ ws).shape

torch.Size([65])

In [36]:
num_epochs = 20
lr = 0.01

In [18]:
torch.randn((3, 1))

tensor([[ 0.4210],
        [-1.3533],
        [-0.5501]])

In [37]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(0, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean()
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

14.815048217773438
14.814902305603027
14.814756393432617
14.814613342285156
14.814469337463379
14.814325332641602
14.814179420471191
14.814035415649414
14.813889503479004
14.81374740600586
14.813602447509766
14.813456535339355
14.813313484191895
14.813169479370117
14.81302547454834
14.812881469726562
14.812737464904785
14.812593460083008
14.81244945526123
14.812305450439453


In [39]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=65).float().flatten()
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(0, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([stoi[last_two[0]], stoi[last_two[1]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

"RomFeiBolWeg.emF!.C.Lu.gooeie'o;;m?L\nYeBos!iuqtoiFBp-;uB-fLeur-ee\neeMCBeo.cuJ-rfLe;esr-sQBre \nFLsEgrr\nLqeur;?3eBU e:mFRreb.eeoeFBzrsrqe.ee-BaY.Nheie\neErueIiirugqe!\nCessCFlFqRrUmB.Lme-i;q;eqL.BFZeLTqTIqBi?:ssIT;rBhKBrl.BEeOqmiMmsLiFez&R.&W;O\n.eeeeBeeTofeogBL-FNqf.eqgwoeej?.-uLBC.FL.-gL\nek;o;WeeelusrRmrh.q-e-H;rLr\nZ'Wee;eqeMoFeZF;q;LE.BmL'Z-LJ..eB;eJeBsreBrLyeFUel.Uuqqqs.LqeehxFLerf;evieeergm.b'fuee.-e&Qe;rLiiieeR-zoerBeLuoLe;i-qrCW;e:keqYhgree.YeBe.qOLxLhef,YeLtBrfoqsmB&FBnFi,iFs-LLe-Ux.qqeqi.OgFfuZ"